# Importing Libraries & Reading JSON

In [56]:
import sys
import os
import subprocess
import json
import itertools

json_list = json.load(open('parameters_json_file.json', 'r'))
print("parameters_json_file.json just opened")

data = json_list["data"]
parameters = json_list["parameters"]
fixed_parameters = parameters["fixed"]
iterable_parameters = parameters["iterable"]
print(iterable_parameters)

parameters_json_file.json just opened
{'rnn_dim': [100, 150, 200, 300], 'char_rnn': [25, 50], 'word_min_freq': [0, 5, 10], 'train_batch_size': [4, 8, 16]}


## Parameters Combinations

In [40]:
listofvalues=iterable_parameters.values()
args_list = []
for it_key in iterable_parameters:
    print(it_key, "\t", iterable_parameters[it_key])
    args_list.append(it_key)
parameter_combinations = list(itertools.product(*listofvalues))
print("\nNumber of combinations to try:", len(parameter_combinations))


rnn_dim 	 [100, 150, 200, 300]
char_rnn 	 [25, 50]
word_min_freq 	 [0, 5, 10]
train_batch_size 	 [4, 8, 16]

Number of combinations to try: 72


## Class: Command - add_arg()

In [41]:
class Command_string:
    def __init__(self, initial_string = "python"):
        self.string = initial_string
    
    def change(self, new_string):
        self.string = new_string
    
    def add_arg_str(self, string):
        self.string += " " + string
        
    def add_arg_dic(self, key, dic):
        self.string += " --" + key + " " + str(dic[key])
    
    def add_arg_val(self, key, val):
        self.string += " --" + key + " " + str(val)

## Commands String

### Fixed string part

In [42]:
fixed_command = Command_string()
fixed_command.add_arg_str(json_list["train.py"])
for path_key in data:
    fixed_command.add_arg_dic(path_key, data)
for fixed_par_key in fixed_parameters:
    fixed_command.add_arg_dic(fixed_par_key, fixed_parameters)
print(fixed_command.string)

python /disks/md0-4T/users/b.mgm/model/train.py --data_path /disks/md0-4T/users/b.mgm/data/CRF --train_path train.bilou_CRF --val_path validation.bilou_CRF --test_path test.bilou_CRF --emb_path /disks/md0-4T/users/b.mgm/data/embedding/pretrained_embedding.txt --char_emb_dim 25 --word_emb_dim 100 --val_batch_size 64 --max_epochs 100


### Changing common part (due to arguments combinations) --> list of strings

In [43]:
common_list = []
tmp_command = Command_string()
name = []
checkpoint_dirs = []

for comb in parameter_combinations:
    tmp_command.change(fixed_command.string)
    comb_num = parameter_combinations.index(comb)
    name.append("training_" + str(comb_num) + "_")
    for it_par_key in iterable_parameters:
        key_index = list(iterable_parameters.keys()).index(it_par_key)
        tmp_command.add_arg_val(it_par_key, comb[key_index])             # adds --parameterName parameterValue
        name[comb_num] += "__" + it_par_key + "_" + str(comb[key_index]) # adds __parameterName_parameterValue
    common_list.append(tmp_command.string)

### Showing one command

In [44]:
print("List length:", len(common_list))
index_to_show = 3
print("List command %d (%d/%d):\n%s" % (index_to_show, index_to_show+1, len(common_list), common_list[index_to_show]))

List length: 72
List command 3 (4/72):
python /disks/md0-4T/users/b.mgm/model/train.py --data_path /disks/md0-4T/users/b.mgm/data/CRF --train_path train.bilou_CRF --val_path validation.bilou_CRF --test_path test.bilou_CRF --emb_path /disks/md0-4T/users/b.mgm/data/embedding/pretrained_embedding.txt --char_emb_dim 25 --word_emb_dim 100 --val_batch_size 64 --max_epochs 100 --rnn_dim 100 --char_rnn 25 --word_min_freq 5 --train_batch_size 4


## Training command

In [45]:
train_command = Command_string()
train_list = common_list.copy()
for comb in parameter_combinations:
    train_command.change("")
    comb_num = parameter_combinations.index(comb)
    train_command.add_arg_val("checkpoint", json_list["checkpoints"] + "/" + name[comb_num])
    train_command.add_arg_str(" | tee -a " + json_list["logs"] + "/" + name[comb_num] + ".log")
    train_list[comb_num] += train_command.string

In [46]:
print(train_command.string)
print("List length:", len(train_list))
index_to_show = 71
print("List command %d (%d/%d):\n%s" % (index_to_show, index_to_show+1, len(train_list), train_list[index_to_show]))


print("List command %d (%d/%d):\n%s" % (index_to_show, index_to_show+1, len(common_list), common_list[index_to_show]))


 --checkpoint /disks/md0-4T/users/b.mgm/checkpoints/training_71___rnn_dim_300__char_rnn_50__word_min_freq_10__train_batch_size_16  | tee -a /disks/md0-4T/users/b.mgm/logs/training_71___rnn_dim_300__char_rnn_50__word_min_freq_10__train_batch_size_16.log
List length: 72
List command 71 (72/72):
python /disks/md0-4T/users/b.mgm/model/train.py --data_path /disks/md0-4T/users/b.mgm/data/CRF --train_path train.bilou_CRF --val_path validation.bilou_CRF --test_path test.bilou_CRF --emb_path /disks/md0-4T/users/b.mgm/data/embedding/pretrained_embedding.txt --char_emb_dim 25 --word_emb_dim 100 --val_batch_size 64 --max_epochs 100 --rnn_dim 300 --char_rnn 50 --word_min_freq 10 --train_batch_size 16 --checkpoint /disks/md0-4T/users/b.mgm/checkpoints/training_71___rnn_dim_300__char_rnn_50__word_min_freq_10__train_batch_size_16  | tee -a /disks/md0-4T/users/b.mgm/logs/training_71___rnn_dim_300__char_rnn_50__word_min_freq_10__train_batch_size_16.log
List command 71 (72/72):
python /disks/md0-4T/users

## Test command

In [68]:
path = "~/work/checkpoints"+"/"+name[1]
print(path)
os.chdir(path)
pwd = subprocess.check_output("pwd")
print(pwd.decode("utf-8"))
cmd = json_list["sort_chekpoints_path"] + " -H " + str(json_list["N_best"]) + " " \
      + json_list["checkpoints"] + "/" + name[1] + " | cut -f 3"
print(cmd)
best_epochs = subprocess.check_output(cmd)
print(best_epochs.decode("utf-8"))

~/work/checkpoints/training_1___rnn_dim_100__char_rnn_25__word_min_freq_0__train_batch_size_8


FileNotFoundError: [Errno 2] No such file or directory: '~/work/checkpoints/training_1___rnn_dim_100__char_rnn_25__word_min_freq_0__train_batch_size_8'

In [ ]:
train_command = Command_string("")
train_list = common_list.copy()
for comb in parameter_combinations:
    train_command.change("")
    comb_num = parameter_combinations.index(comb)
    train_command.add_arg_val("checkpoint", json_list["checkpoints"] + "/" + name[comb_num])
    train_command.add_arg_str(" |& tee -a " + json_list["logs"] + "/" + name[comb_num] + ".log")
    train_list[comb_num] += train_command.string

# Output Files

## Train File

In [ ]:
#Opening and writing in train file
train_file = open(json_list["output_train"], 'w')
for line in train_list:
    train_file.write(line + "\n")
train_file.close()

In [36]:
train_file = open(json_list["output_train"], 'r')
print("Train file:", json_list["output_train"], "\n\n", train_file.read())
train_file.close()

Train file: training_commands.lst 

 python /disks/md0-4T/users/b.mgm/model/train.py --data_path /disks/md0-4T/users/b.mgm/data/CRF --train_path train.bilou_CRF --val_path validation.bilou_CRF --test_path test.bilou_CRF --emb_path /disks/md0-4T/users/b.mgm/data/embedding/pretrained_embedding.txt --char_emb_dim 25 --word_emb_dim 100 --val_batch_size 64 --max_epochs 100 --rnn_dim 100 --char_rnn 25 --word_min_freq 0 --train_batch_size 4 --checkpoint /disks/md0-4T/users/b.mgm/checkpoints/training_0___rnn_dim_100__char_rnn_25__word_min_freq_0__train_batch_size_4  | tee -a /disks/md0-4T/users/b.mgm/logs/training_0___rnn_dim_100__char_rnn_25__word_min_freq_0__train_batch_size_4.log
python /disks/md0-4T/users/b.mgm/model/train.py --data_path /disks/md0-4T/users/b.mgm/data/CRF --train_path train.bilou_CRF --val_path validation.bilou_CRF --test_path test.bilou_CRF --emb_path /disks/md0-4T/users/b.mgm/data/embedding/pretrained_embedding.txt --char_emb_dim 25 --word_emb_dim 100 --val_batch_size 64

## Test file

In [34]:
test_file = open(json_list["output_test"], 'w')
for line in test_list:
    test_file.write(line + "\n")
test_file.close()

Train file: training_commands.lst
Train file: training_commands.lst


In [38]:
test_file = open(json_list["output_test"], 'r')
print("Test file:", json_list["output_test"], "\n\n", test_file.read())
test_file.close()

FileNotFoundError: [Errno 2] No such file or directory: 'test_commands.lst'

In [22]:
name = []
for i in range(0,10):
    name.append(str(i))
name2 = name.copy()
name2.sort(reverse=True)
print(name, name2)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] ['9', '8', '7', '6', '5', '4', '3', '2', '1', '0']
